In [1]:
import bilby
import sys
sys.path.append('../')
import numpy as np
from bilby.core.utils import ra_dec_to_theta_phi, speed_of_light
from bilby.core.utils.constants import *
import bilby.gw.utils as gwutils
from copy import deepcopy
from scipy.stats import multivariate_normal
from source import *
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'source'

In [ ]:
data = np.loadtxt('samples_SNR_CE_CE20.txt')

# Output array for samples + SNR
output_CE_CE20 = []
snrcut_CE = 10
minimum_frequency = 5
sampling_frequency = 4096
reference_frequency = 100
tc_offset = 1
waveformname = "IMRPhenomD"

e_dict = dict(
    chirp_mass=1e-7, mass_ratio=1e-5, chi_1=1e-5, chi_2=1e-5,
    ra=1e-5, dec=1e-5, luminosity_distance = 1e-5,
    theta_jn=1e-5, psi=1e-5, phase=1e-5, geocent_time=1e-5, A = 1e-5
)

#ifos = bilby.gw.detector.InterferometerList(['CE', 'CE20'])
#ifo_CE, ifo_CE20 = ifos[0], ifos[1]

for i, row in enumerate(data):
    if i % 100 == 0:
        print(i)
    m1, m2, ra, dec, psi, luminosity_distance, theta_jn, phase, chi_1, chi_2, geocent_time, snr_CE, snr_CE20 = row
    if snr_CE < snrcut_CE:
        continue
    #log10_luminosity_distance = np.log10(luminosity_distance)

    q = bilby.gw.conversion.component_masses_to_mass_ratio(m1, m2)
    z = bilby.gw.conversion.luminosity_distance_to_redshift(luminosity_distance)
    detector_frame_chirp_mass = (1+z)*bilby.gw.conversion.component_masses_to_chirp_mass(m1, m2)

    ifos = bilby.gw.detector.InterferometerList(['CE', 'CE20'])
    ifo_CE, ifo_CE20 = ifos[0], ifos[1]
    

    chirp_mass_in_seconds = detector_frame_chirp_mass * solar_mass * gravitational_constant / speed_of_light**3.
    t0 = -5. / 256. * chirp_mass_in_seconds * (np.pi * chirp_mass_in_seconds * minimum_frequency)**(-8. / 3.)
    duration = 2**(np.int32(np.log2(np.abs(t0)))+1)
    start_time = geocent_time - duration + tc_offset

    waveform_generator = bilby.gw.WaveformGenerator(
        duration=duration, sampling_frequency=sampling_frequency,
        frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
        parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
        waveform_arguments=dict(waveform_approximant=waveformname, reference_frequency=reference_frequency, minimum_frequency=minimum_frequency))

    injection_parameters = dict(
    chirp_mass=detector_frame_chirp_mass,
    mass_ratio=q,
    chi_1=chi_1, chi_2=chi_2,
    ra=ra, dec=dec,
    luminosity_distance=luminosity_distance,  
    theta_jn=theta_jn,
    psi=psi,
    phase=phase,
    geocent_time=geocent_time,
    a=0, A=0)

    frequencies, h_CE, snr_CE = calc_waveform_and_snr(ifo_CE, start_time, minimum_frequency, injection_parameters, waveform_generator)
    frequencies, h_CE20, snr_CE20 = calc_waveform_and_snr(ifo_CE20, start_time, minimum_frequency, injection_parameters, waveform_generator)
    #frequencies_A1, h_A1, snr_A1 = calc_waveform_and_snr(ifo_A1, start_time, minimum_frequency, injection_parameters, waveform_generator)
    
    Fisher_CE = calc_Fisher(h_CE, frequencies, ifo_CE, start_time, minimum_frequency, injection_parameters, waveform_generator, [True,True,True], e_dict, skip=['a', 'H0'])
    Fisher_CE20 = calc_Fisher(h_CE20, frequencies, ifo_CE20, start_time, minimum_frequency, injection_parameters, waveform_generator, [True,True,True], e_dict, skip=['a', 'H0'])
    #Fisher_A1 = calc_Fisher(h_A1, frequencies_A1, ifo_A1, start_time, minimum_frequency, injection_parameters, waveform_generator, [True,True,True], e_dict, skip=['a', 'H0'])

    #np.savez_compressed('output/'+str(i)+'.npz', Fisher_CE=Fisher_CE, Fisher_CE20=Fisher_CE20)
    #np.savez_compressed(f'output/{i}.npz', Fisher_CE=Fisher_CE, Fisher_CE20=Fisher_CE20)

    import os
    # Create output folder if not exists
    os.makedirs('output_CE_CE20', exist_ok=True)

    # Save Fisher matrices
    np.savez_compressed(f'output_CE_CE20/{i}.npz', Fisher_CE=Fisher_CE, Fisher_CE20=Fisher_CE20)


In [ ]:
data = np.loadtxt('samples_SNR_A1.txt')

# Output array for samples + SNR
output_A1 = []
snrcut_CE = 10
minimum_frequency = 15
sampling_frequency = 4096
reference_frequency = 100
tc_offset = 1
waveformname = "IMRPhenomD"

e_dict = dict(
    chirp_mass=1e-7, mass_ratio=1e-5, chi_1=1e-5, chi_2=1e-5,
    ra=1e-5, dec=1e-5, luminosity_distance = 1e-5,
    theta_jn=1e-5, psi=1e-5, phase=1e-5, geocent_time=1e-5, A = 1e-5
)

#ifos = bilby.gw.detector.InterferometerList(['CE', 'CE20'])
#ifo_CE, ifo_CE20 = ifos[0], ifos[1]

for i, row in enumerate(data):
    if i % 100 == 0:
        print(i)
    m1, m2, ra, dec, psi, luminosity_distance, theta_jn, phase, chi_1, chi_2, geocent_time, snr_A1 = row
    #log10_luminosity_distance = np.log10(luminosity_distance)

    q = bilby.gw.conversion.component_masses_to_mass_ratio(m1, m2)
    z = bilby.gw.conversion.luminosity_distance_to_redshift(luminosity_distance)
    detector_frame_chirp_mass = (1+z)*bilby.gw.conversion.component_masses_to_chirp_mass(m1, m2)

    ifos = bilby.gw.detector.InterferometerList(['A1'])
    ifo_A1 = ifos[0]
    

    chirp_mass_in_seconds = detector_frame_chirp_mass * solar_mass * gravitational_constant / speed_of_light**3.
    t0 = -5. / 256. * chirp_mass_in_seconds * (np.pi * chirp_mass_in_seconds * minimum_frequency)**(-8. / 3.)
    duration = 2**(np.int32(np.log2(np.abs(t0)))+1)
    start_time = geocent_time - duration + tc_offset

    waveform_generator = bilby.gw.WaveformGenerator(
        duration=duration, sampling_frequency=sampling_frequency,
        frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
        parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
        waveform_arguments=dict(waveform_approximant=waveformname, reference_frequency=reference_frequency, minimum_frequency=minimum_frequency))

    injection_parameters = dict(
    chirp_mass=detector_frame_chirp_mass,
    mass_ratio=q,
    chi_1=chi_1, chi_2=chi_2,
    ra=ra, dec=dec,
    luminosity_distance=luminosity_distance,  
    theta_jn=theta_jn,
    psi=psi,
    phase=phase,
    geocent_time=geocent_time,
    a=0, A=0)

    #frequencies, h_CE, snr_CE = calc_waveform_and_snr(ifo_CE, start_time, minimum_frequency, injection_parameters, waveform_generator)
    #frequencies, h_CE20, snr_CE20 = calc_waveform_and_snr(ifo_CE20, start_time, minimum_frequency, injection_parameters, waveform_generator)
    frequencies_A1, h_A1, snr_A1 = calc_waveform_and_snr(ifo_A1, start_time, minimum_frequency, injection_parameters, waveform_generator)
    
    #Fisher_CE = calc_Fisher(h_CE, frequencies, ifo_CE, start_time, minimum_frequency, injection_parameters, waveform_generator, [True,True,True], e_dict, skip=['a', 'H0'])
    #Fisher_CE20 = calc_Fisher(h_CE20, frequencies, ifo_CE20, start_time, minimum_frequency, injection_parameters, waveform_generator, [True,True,True], e_dict, skip=['a', 'H0'])
    Fisher_A1 = calc_Fisher(h_A1, frequencies_A1, ifo_A1, start_time, minimum_frequency, injection_parameters, waveform_generator, [True,True,True], e_dict, skip=['a', 'H0'])

    #np.savez_compressed('output/'+str(i)+'.txt', Fisher_A1=Fisher_A1)
    #np.savez_compressed(f'output/{i}.npz', Fisher_CE=Fisher_CE, Fisher_CE20=Fisher_CE20)

    import os
    # Create output folder if not exists
    os.makedirs('output_A1', exist_ok=True)

    # Save Fisher matrices
    np.savez_compressed(f'output_A1/{i}.npz', Fisher_A1=Fisher_A1)


In [ ]:
data = np.loadtxt('samples_SNR_CE_CE20.txt')

output = []
sigma_A_CE = []
sigma_A_CECE20 = []
for snrcut_ in range(50, 1000, 10):
    sigma_A_inv_squared_CE, sigma_A_inv_squared_CECE20 = 0,0
    for i, row in enumerate(data):
        m1, m2, ra, dec, psi, luminosity_distance, theta_jn, phase, chi_1, chi_2, geocent_time, snr_CE, snr_CE20 = row
        if snr_CE < snrcut_:
            continue
        else:
            Fisher = np.load(f'output_CE_CE20/{i}.npz')
            Fisher_CE = Fisher['Fisher_CE']
            Fisher_CE = remove(Fisher_CE, [8,9]) # fixing the phase
            Fisher_CE20 = Fisher['Fisher_CE20']
            Fisher_CE20 = remove(Fisher_CE20, [8,9])
            
            Fisher_CECE20 = Fisher_CE + Fisher_CE20
            
            sigma_A_inv_squared_CE += 1/np.linalg.inv(Fisher_CE)[-1, -1]
            sigma_A_inv_squared_CECE20 += 1/np.linalg.inv(Fisher_CECE20)[-1, -1]

    epsilon = 1e-12  # small number to avoid division by zero
    sigma_A_CE.append(np.sqrt(1/(sigma_A_inv_squared_CE + epsilon)))
    sigma_A_CECE20.append(np.sqrt(1/(sigma_A_inv_squared_CECE20 + epsilon)))

sigma_A_CE = np.array(sigma_A_CE)
sigma_A_CECE20 = np.array(sigma_A_CECE20)

In [ ]:
data = np.loadtxt('samples_SNR_A1.txt')

output = []
sigma_A_CE20A1 = []
sigma_A_CECE20A1 = []
for snrcut_ in range(50, 1000, 10):
    sigma_A_inv_squared_CE, sigma_A_inv_squared_CECE20, sigma_A_inv_squared_CE20A1, sigma_A_inv_squared_CECE20A1 = 0,0,0,0
    for i, row in enumerate(data):
        m1, m2, ra, dec, psi, luminosity_distance, theta_jn, phase, chi_1, chi_2, geocent_time, snr_A1 = row
        if snr_CE < snrcut_:
            continue
        else:
            Fisher = np.load(f'output_A1/{i}.npz')
            Fisher_A1 = Fisher['Fisher_A1']
            Fisher_A1 = remove(Fisher_A1, [8,9])
            
            Fisher_CE20A1 = Fisher_CE20 + Fisher_A1
            Fisher_CECE20A1 = Fisher_CECE20 + Fisher_A1

            sigma_A_inv_squared_CE20A1 += 1/np.linalg.inv(Fisher_CE20A1)[-1, -1]
            sigma_A_inv_squared_CECE20A1 += 1/np.linalg.inv(Fisher_CECE20A1)[-1, -1]

    epsilon = 1e-12  # small number to avoid division by zero
    sigma_A_CE20A1.append(np.sqrt(1/(sigma_A_inv_squared_CE20A1 + epsilon)))
    sigma_A_CECE20A1.append(np.sqrt(1/(sigma_A_inv_squared_CECE20A1 + epsilon)))

sigma_A_CE20A1 = np.array(sigma_A_CE20A1)
sigma_A_CECE20A1 = np.array(sigma_A_CECE20A1)

In [ ]:
#plt.plot(sigma_A_CE, label='CE')
plt.fill_between(range(50, 1000, 10), -10*sigma_A_CE, 10*sigma_A_CE, alpha=0.3, label='CE')
plt.fill_between(range(50, 1000, 10), -5*sigma_A_CECE20, 5*sigma_A_CECE20, alpha=0.3, label='CE+CE20')
plt.fill_between(range(50, 1000, 10), -2*sigma_A_CE20A1, 2*sigma_A_CE20A1, alpha=0.3, label='CE20+A1')
plt.fill_between(range(50, 1000, 10), -1*sigma_A_CECE20A1, 1*sigma_A_CECE20A1, alpha=0.3, label='CE+CE20+A1')
plt.xscale('log')
plt.axhline(0, color='black', lw=1, label='Injected')
plt.xlabel('Selecting events with SNR (at CE) >')
plt.ylabel('$A_0[\\times 10^{-21} peV^{2}]$')
plt.grid()
plt.legend()
